In [1]:
import numpy as np
import matplotlib.pyplot as plt
import metropolishastings as mh
from rhat_n_eff import rhat_n_eff

In [2]:
def myprocess(state):
    next_state = np.random.randint(1, 4+1)
    return next_state, 0.25, 0.25

In [3]:
def weight(state):
    w = {
        1: 20, 
        2: 8,
        3: 3,
        4: 1,
    }
    return w[state]

Probabilites

In [4]:
prob = np.array([20, 8, 3, 1])
prob = prob / prob.sum()
prob

array([0.625  , 0.25   , 0.09375, 0.03125])

In [5]:
def indicators(one_draw):
    #result = 1 if one_draw == 1 else 0
    result = [1 if (one_draw == i) else 0 for i in [1, 2, 3, 4]]
    return result

In [6]:
initialStates = [1, 2, 3, 4]
numberOfDraws = 100000
maxNumberOfIterations = 10

In [7]:
mh.MetropolisHastings(
    initialStates,
    myprocess,
    weight,
    indicators,
    numberOfDraws,
    maxNumberOfIterations,
)

Before draws.shape=(4, 100000, 4)
After draws.shape=(4, 4, 100000)
RICO: rico_r=1.000020178426886 rico_n=98473.21961783161
MICHEL: R_hat=1.000020178426886 neff=98473.21961783174
RICO: rico_r=1.0000201942138456 rico_n=123279.55002306009
MICHEL: R_hat=1.0000201942138456 neff=123279.55002306035
RICO: rico_r=1.0000238847138394 rico_n=174501.07406927733
MICHEL: R_hat=1.0000238847138394 neff=174501.07406927733
RICO: rico_r=1.0000069686959312 rico_n=234077.08856384055
MICHEL: R_hat=1.0000069686959312 neff=234077.08856384055


(array([0, 1, 1, ..., 0, 0, 0]),
 array([0.6238475, 0.2512475, 0.0935625, 0.0313425]),
 True,
 0)

In [8]:
prob

array([0.625  , 0.25   , 0.09375, 0.03125])